In this notebook, we are going to prepare the data to be used in future models.

In [1]:
import numpy as np                       
import xarray as xr                      
import pandas as pd     
import cftime           
import zipfile       
from datetime import datetime, timedelta
import os
import sys

# Temperature

## Reanalysis

In [2]:
reanalysis_nc = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2000a2010_uy.nc')
reanalysis_nc_2 = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2011a2021_uy.nc')

def transform_reanalysis(data_nc):
    reanalysis = pd.DataFrame()
    reanalysis['time'] = pd.to_datetime(data_nc.time)
    reanalysis['tas'] = data_nc.t2m.mean(dim=["latitude","longitude"]) 
    reanalysis = reanalysis.loc[reanalysis["time"] < datetime.fromisoformat('2015-01-01T00:00:00')]
    reanalysis['date'] = reanalysis['time'].dt.date
    reanalysis['day'] = reanalysis['time'].dt.day
    reanalysis['month'] = reanalysis['time'].dt.month
  
    ## TODO: Max and min has their own model, but for now we will calculate them here. But we should load them from their model.
    reanalysis_by_day = reanalysis[["date", "tas"]].groupby('date') \
                                                    .agg({'tas': ['mean', 'min', 'max']})
    reanalysis_by_day.columns = ['_'.join(col).strip() for col in reanalysis_by_day.columns.values]
    reanalysis_by_day = reanalysis_by_day.rename(columns={'tas_mean': 'tas_daily', 'tas_min': 'tas_min_daily', 'tas_max': 'tas_max_daily'})
    ########################################################
  
    reanalysis = reanalysis.join(reanalysis_by_day, on="date") \
                            .drop('date', axis=1)
    return reanalysis

reanalysis_tas = pd.concat([transform_reanalysis(reanalysis_nc), transform_reanalysis(reanalysis_nc_2)])
reanalysis_tas = reanalysis_tas[((reanalysis_tas['day'] != 29) | (reanalysis_tas['month'] != 2))] #Delete all Feb 29th 
reanalysis_tas = reanalysis_tas[['time', 'tas_daily','tas_min_daily','tas_max_daily','tas']] #select the variables we want to keep
reanalysis_tas[['time', 'tas_daily','tas']] .to_csv('reanalysis_tas_training_data.csv',  index=False)

## CMIP

In [3]:
CESM2_WACCM_nc = xr.open_dataset('../data/tas_day_CESM2-WACCM_historical_r1i1p1f1_gn_20000101-20141231_v20190227.nc')
CESM2_WACCM = pd.DataFrame()
CESM2_WACCM['time'] = CESM2_WACCM_nc.time
CESM2_WACCM['CESM2_WACCM'] = CESM2_WACCM_nc.tas.mean(dim=["lat","lon"])
CESM2_WACCM['time'] = CESM2_WACCM['time'].apply(lambda x: datetime.fromisoformat(x.isoformat()))

CESM2_WACCM_by_hour = []

for row in CESM2_WACCM.values:
    for h in range(0,24):
        new_row = pd.Series({
            'time': row[0] + timedelta(hours=h),
            'tas': 0,
            'tas_daily': row[1]
        })
        CESM2_WACCM_by_hour.append(new_row)

CESM2_WACCM = pd.DataFrame(CESM2_WACCM_by_hour)
CESM2_WACCM.to_csv('cmip6_tas_to_downscale_data.csv', index=False)

# Wind Speed

## Reanalysis

**Note**: I've assumed that sfcwind = sqrt(u^2 + v^2). Corroboration is needed.

In [4]:
reanalysis_wind_U_nc = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_U10m_horario_2000a2010_uy.nc')
reanalysis_wind_V_nc = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_V10m_horario_2000a2010_uy.nc')
reanalysis_wind_U_2_nc = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_U10m_horario_2011a2021_uy.nc')
reanalysis_wind_V_2_nc = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_V10m_horario_2011a2021_uy.nc')


def transform_reanalysis_wind(U_nc, V_nc):
    reanalysis = pd.DataFrame()
    reanalysis['time'] = pd.to_datetime(U_nc.time)
    reanalysis['sfcWind'] = np.sqrt(np.square(U_nc.u10) + np.square(V_nc.v10)) \
                               .mean(dim=["latitude","longitude"]) 
    reanalysis = reanalysis.loc[reanalysis["time"] < datetime.fromisoformat('2015-01-01T00:00:00')]
    reanalysis['date'] = reanalysis['time'].dt.date
    reanalysis['day'] = reanalysis['time'].dt.day
    reanalysis['month'] = reanalysis['time'].dt.month
    reanalysis_by_day = reanalysis[["date", "sfcWind"]].groupby('date') \
                                                   .mean() \
                                                   .rename(columns={"sfcWind":"sfcWind_daily"})
    reanalysis = reanalysis.join(reanalysis_by_day, on="date") \
                            .drop('date', axis=1)
    return reanalysis

reanalysis_wind = pd.concat([transform_reanalysis_wind(reanalysis_wind_U_nc, reanalysis_wind_V_nc), 
                             transform_reanalysis_wind(reanalysis_wind_U_2_nc, reanalysis_wind_V_2_nc)])

reanalysis_wind = reanalysis_wind[((reanalysis_wind['day'] != 29) | (reanalysis_wind['month'] != 2))] #Delete all Feb 29th 
reanalysis_wind = reanalysis_wind[['time', 'sfcWind_daily', 'sfcWind']] #select the variables we want to keep
reanalysis_wind.to_csv('reanalysis_sfcWind_training_data.csv',  index=False)

### CMIP

In [5]:
CESM2_WACCM_wind_nc = xr.open_dataset('../data/sfcWind_day_CESM2-WACCM_historical_r1i1p1f1_gn_20000101-20141231_v20190227.nc')
CESM2_WACCM_wind = pd.DataFrame()
CESM2_WACCM_wind['time'] = CESM2_WACCM_wind_nc.time
CESM2_WACCM_wind['CESM2_WACCM'] = CESM2_WACCM_wind_nc.sfcWind.mean(dim=["lat","lon"])
CESM2_WACCM_wind['time'] = CESM2_WACCM_wind['time'].apply(lambda x: datetime.fromisoformat(x.isoformat()))

CESM2_WACCM_wind_by_hour = []

for row in CESM2_WACCM_wind.values:
    for h in range(0,24):
        new_row = pd.Series({
            'time': row[0] + timedelta(hours=h),
            'sfcWind': 0,
            'sfcWind_daily': row[1]
        })
        CESM2_WACCM_wind_by_hour.append(new_row)

CESM2_WACCM_wind = pd.DataFrame(CESM2_WACCM_wind_by_hour)
CESM2_WACCM_wind.to_csv('cmip6_sfcWind_to_downscale_data.csv', index=False)
#CESM2_WACCM_wind.head()

# Precipitation

## Reanalysis

In [6]:
PRECIPTITATION_FOLDER = "../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis-precipitation/"

def transform_reanalysis(data_nc):
    reanalysis_precipitation = pd.DataFrame()
    try:
        reanalysis_precipitation['time'] = pd.to_datetime(data_nc.time)
        reanalysis_precipitation['pr'] = data_nc.tp.mean(dim=["latitude","longitude"])*1000
    except:
        return reanalysis_precipitation
    reanalysis_precipitation['date'] = reanalysis_precipitation['time'].dt.date
    reanalysis_precipitation['day'] = reanalysis_precipitation['time'].dt.day
    reanalysis_precipitation['month'] = reanalysis_precipitation['time'].dt.month
    reanalysis_precipitation_by_day = reanalysis_precipitation[["date", "pr"]].groupby('date') \
                                                   .mean() \
                                                   .rename(columns={"pr":"pr_daily"})
    reanalysis_precipitation = reanalysis_precipitation.join(reanalysis_precipitation_by_day, on="date") \
                            .drop('date', axis=1)
    return reanalysis_precipitation

reanalysis_precipitation = pd.DataFrame()

for file in os.listdir(PRECIPTITATION_FOLDER):

    with zipfile.ZipFile(PRECIPTITATION_FOLDER + file,"r") as zip_ref:
        zip_ref.extractall(PRECIPTITATION_FOLDER + "data")
    
    precipitation_nc = xr.open_dataset(PRECIPTITATION_FOLDER + "data/data.nc")
    df = transform_reanalysis(precipitation_nc)
    reanalysis_precipitation = pd.concat([reanalysis_precipitation, df])
    os.remove(PRECIPTITATION_FOLDER + "data/data.nc")
    os.removedirs(PRECIPTITATION_FOLDER + "data")

reanalysis_precipitation.sort_values(by='time', inplace=True)
reanalysis_precipitation.reset_index(drop=True, inplace=True)
reanalysis_precipitation = reanalysis_precipitation.loc[reanalysis_precipitation["time"] < datetime.fromisoformat('2015-01-01T00:00:00')]
reanalysis_precipitation = reanalysis_precipitation[((reanalysis_precipitation['day'] != 29) | (reanalysis_precipitation['month'] != 2))] #Delete all Feb 29th 
reanalysis_precipitation = reanalysis_precipitation[['time', 'pr_daily', 'pr']] #select the variables we want to keep
reanalysis_precipitation.to_csv('reanalysis_precipitation_training_data.csv',  index=False)

### CMIP

In [7]:
CESM2_WACCM_precipitation_nc = xr.open_dataset('../data/pr_day_CESM2-WACCM_historical_r1i1p1f1_gn_20000101-20141231_v20190415.nc')
CESM2_WACCM_precipitation = pd.DataFrame()
CESM2_WACCM_precipitation['time'] = CESM2_WACCM_precipitation_nc.time
CESM2_WACCM_precipitation['CESM2_WACCM'] = CESM2_WACCM_precipitation_nc.pr.mean(dim=["lat","lon"])*3600
CESM2_WACCM_precipitation['time'] = CESM2_WACCM_precipitation['time'].apply(lambda x: datetime.fromisoformat(x.isoformat()))

CESM2_WACCM_precipitation_by_hour = []

for row in CESM2_WACCM_precipitation.values:
    for h in range(0,24):
        new_row = pd.Series({
            'time': row[0] + timedelta(hours=h),
            'pr': 0,
            'pr_daily': row[1]
        })
        CESM2_WACCM_precipitation_by_hour.append(new_row)

CESM2_WACCM_precipitation = pd.DataFrame(CESM2_WACCM_precipitation_by_hour)
CESM2_WACCM_precipitation.to_csv('cmip6_precipitation_to_downscale_data.csv', index=False)

# Surface Pressure

## Reanalysis

In [8]:
SURFACE_PRESSURE_FOLDER = "../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis-surface_pressure/"

def transform_reanalysis_surface_pressure(data_nc):
    reanalysis_surface_pressure = pd.DataFrame()
    try:
        reanalysis_surface_pressure['time'] = pd.to_datetime(data_nc.time)
        reanalysis_surface_pressure['sp'] = data_nc.sp.mean(dim=["latitude","longitude"])
    except:
        return reanalysis_surface_pressure
    reanalysis_surface_pressure['date'] = reanalysis_surface_pressure['time'].dt.date
    reanalysis_surface_pressure['day'] = reanalysis_surface_pressure['time'].dt.day
    reanalysis_surface_pressure['month'] = reanalysis_surface_pressure['time'].dt.month
    reanalysis_surface_pressure_by_day = reanalysis_surface_pressure[["date", "sp"]].groupby('date') \
                                                   .mean() \
                                                   .rename(columns={"sp":"sp_daily"})
    reanalysis_surface_pressure = reanalysis_surface_pressure.join(reanalysis_surface_pressure_by_day, on="date") \
                            .drop('date', axis=1)
    return reanalysis_surface_pressure

reanalysis_surface_pressure = pd.DataFrame()

for file in os.listdir(SURFACE_PRESSURE_FOLDER):

    with zipfile.ZipFile(SURFACE_PRESSURE_FOLDER + file,"r") as zip_ref:
        zip_ref.extractall(SURFACE_PRESSURE_FOLDER + "data")
    
    surface_pressure_nc = xr.open_dataset(SURFACE_PRESSURE_FOLDER + "data/data.nc")
    df = transform_reanalysis_surface_pressure(surface_pressure_nc)
    reanalysis_surface_pressure = pd.concat([reanalysis_surface_pressure, df])
    os.remove(SURFACE_PRESSURE_FOLDER + "data/data.nc")
    os.removedirs(SURFACE_PRESSURE_FOLDER + "data")

reanalysis_surface_pressure.sort_values(by='time', inplace=True)
reanalysis_surface_pressure.reset_index(drop=True, inplace=True)
reanalysis_surface_pressure = reanalysis_surface_pressure[((reanalysis_surface_pressure['day'] != 29) | (reanalysis_surface_pressure['month'] != 2))] #Delete all Feb 29th 
reanalysis_surface_pressure = reanalysis_surface_pressure[['time', 'sp_daily', 'sp']] #select the variables we want to keep
reanalysis_surface_pressure = reanalysis_surface_pressure.loc[reanalysis_surface_pressure["time"] < datetime.fromisoformat('2015-01-01T00:00:00')]
reanalysis_surface_pressure.to_csv('reanalysis_surface_pressure_training_data.csv',  index=False)

# Total Cloud Percentage

## Reanalysis

In [ ]:
reanalysis_nc = xr.open_dataset('../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis - total_cloud_percentage/total_cloud_percentage.nc')

reanalysis_tot_cloud_pctg = transform_reanalysis_tot_cloud_pctg(reanalysis_nc)
#reanalysis_tot_cloud_pctg = reanalysis_tot_cloud_pctg[['time', 'tas_daily','tas_min_daily','tas_max_daily','tas']] #select the variables we want to keep
reanalysis_tot_cloud_pctg[['time', 'tas_daily','tas']].to_csv('reanalysis_tas_training_data.csv',  index=False)

# Join all the data

In [9]:
# Merge all the reanalysis data
reanalysis = reanalysis_tas.merge(reanalysis_wind, on='time', how='inner') \
                            .merge(reanalysis_precipitation, on='time', how='inner') \
                            .merge(reanalysis_surface_pressure, on='time', how='inner')
reanalysis.to_csv('reanalysis.csv', index=False)
reanalysis.head(5)

,time,tas_daily,tas_min_daily,tas_max_daily,tas,sfcWind_daily,sfcWind,pr_daily,pr,sp_daily,sp
0,2000-01-01 00:00:00,295.357391,289.210754,301.250488,295.123260,4.347946,4.526224,0.018645,0.128054,100092.25,100063.445312
1,2000-01-01 01:00:00,295.357391,289.210754,301.250488,294.918762,4.347946,4.780273,0.018645,0.001557,100092.25,100144.429688
2,2000-01-01 02:00:00,295.357391,289.210754,301.250488,294.258972,4.347946,4.837720,0.018645,0.004051,100092.25,100184.609375
3,2000-01-01 03:00:00,295.357391,289.210754,301.250488,291.075500,4.347946,4.704229,0.018645,0.005007,100092.25,100177.500000
4,2000-01-01 04:00:00,295.357391,289.210754,301.250488,290.787567,4.347946,4.446250,0.018645,0.006419,100092.25,100174.578125
